**Студент: Разуев Г.А.**
<br>
**Группа:  КЭ - 403**

# Задание

1. Разработайте программу, которая выполняет поиск частых наборов объектов в заданном наборе данных с помощью алгоритма Apriori (или одной из его модификаций). Список результирующих наборов должен содержать как наборы, так и значение поддержки для каждого набора. Параметрами программы являются набор, порог поддержки и способ упорядочивания результирующего списка наборов (по убыванию значения поддержки или лексикографическое).

2. Проведите эксперименты на наборе данных baskets.csv (сведения о покупках в супермаркете). В экспериментах варьируйте пороговое значение поддержки (например: 1%, 3%, 5%, 10%, 15%).

3. Выполните визуализацию результатов экспериментов в виде следующих диаграмм:

    * сравнение быстродействия на фиксированном наборе данных при изменяемом пороге поддержки;

    * количество частых наборов объектов различной длины на фиксированном наборе данных при изменяемом пороге поддержки.

4. Подготовьте отчет о выполнении задания и загрузите отчет в формате PDF в систему. Отчет должен представлять собой связный и структурированный документ со следующими разделами:  
    * формулировка задания; 
    * гиперссылка на каталог репозитория с исходными текстами, наборами данных и др. сопутствующими материалами; 
    * рисунки с результатами визуализации; 
    * пояснения, раскрывающие смысл полученных результатов.

# 1. Реализация алгоритма

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from typing import List
import os
import time

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
class AprioriSolver:
    def __init__(self):
        pass

    def _get_frequent_itemsets(self, data: List[List], min_support: float, verbose: bool) -> pd.DataFrame:
        """Метод для нахождения частых наборов объектов"""
        # Преобразуем данные в список транзакций
        transactions = [set(lst) for lst in data]
        
        # Установим начальные значения
        item_count = {}
        
        # Подсчитваем частоту одиночных предметов
        for transaction in transactions:
            for item in transaction:
                item_count[item] = item_count.get(item, 0) + 1
                    
        # Фильтруем по порогу поддержки
        transactions_count = len(transactions)
        frequent_itemsets = {
            frozenset([item]): count for item, count in item_count.items() 
            if count / transactions_count >= min_support
        }
        
        set_size = 2  # Начнем с пар
        candidate_itemsets = self._generate_candidates(frequent_itemsets.keys(), set_size)
        frequent_itemsets = {}
        while candidate_itemsets:
            item_count = {}
            
            # Подсчитываем поддержку для кандидатных наборов
            for transaction in transactions:
                for candidate in candidate_itemsets:
                    if candidate.issubset(transaction):
                        item_count[candidate] = item_count.get(candidate, 0) + 1

            # Фильтруем по порогу поддержки
            candidate_itemsets = {
                itemset: count for itemset, count in item_count.items() 
                if count / transactions_count >= min_support
            }
            if verbose:
                print(set_size)
                print(candidate_itemsets)

            # Добавляем новые частые наборы к общему списку
            frequent_itemsets.update(candidate_itemsets)

            set_size += 1  # Переходим к следующему размеру наборов
            # Генерируем наборы кандидатных наборов
            candidate_itemsets = self._generate_candidates(frequent_itemsets.keys(), set_size)

        return frequent_itemsets

    def _generate_candidates(self, frequent_itemsets: set, set_size: int) -> set:
        """Метод для генерации кандидатных наборов"""
        candidates = set()
        itemsets_list = list(frequent_itemsets)
        for i in range(len(itemsets_list)):
            for j in range(i + 1, len(itemsets_list)):
                # Объединяем два набора, если первые (set_size-1) элементы совпадают
                candidate = itemsets_list[i] | itemsets_list[j]
                if len(candidate) == set_size:
                    candidates.add(candidate)
        return candidates

    def solve(self, data: List[List], min_support: float, sort_method: str = 'support', verbose: bool = False) -> pd.DataFrame:
        """Метод для запуска алгоритма"""
        # Исходные частые наборы
        frequent_itemsets = self._get_frequent_itemsets(data, min_support, verbose)

        # Преобразуем результаты в DataFrame
        results = [(tuple(sorted(itemset)), count) for itemset, count in frequent_itemsets.items()]
        results_df = pd.DataFrame(results, columns=['items_set', 'support'])

        # Рассчитываем поддержку
        results_df['support'] = results_df['support'] / len(data)

        # Упорядочиваем результаты 
        if sort_method == 'support':
            results_df = results_df.sort_values(by='support', ascending=False)
        elif sort_method == 'lexical':
            results_df = results_df.sort_values(by='items_set')
        else:
            raise ValueError('Invalid sort method')
        
        return results_df.reset_index(drop=True)

In [3]:
data = [
    ['I1', 'I2', 'I5'],
    ['I2', 'I4'],
    ['I2', 'I3'],
    ['I1', 'I2', 'I4'],
    ['I1', 'I3'],
    ['I2', 'I3'],
    ['I1', 'I3'],
    ['I1', 'I2', 'I3', 'I5'],
    ['I1', 'I2', 'I3'],
    ['I6']
]

apriori_solver = AprioriSolver()
min_support = 0.  # Порог поддержки
result = apriori_solver.solve(data, min_support, sort_method='lexical', verbose=False)
result

,items_set,support
0,"(I1, I2)",0.4
1,"(I1, I2, I3)",0.2
2,"(I1, I2, I3, I5)",0.1
3,"(I1, I2, I4)",0.1
4,"(I1, I2, I5)",0.2
5,"(I1, I3)",0.4
6,"(I1, I3, I5)",0.1
7,"(I1, I4)",0.1
8,"(I1, I5)",0.2
9,"(I2, I3)",0.4


# 2. Эксперименты

In [4]:
import chardet

In [5]:
DATA_PATH = 'data'

In [6]:
with open(os.path.join(DATA_PATH, 'baskets.csv'), 'rb') as f:
    result = chardet.detect(f.read())

print(result['encoding'])

windows-1251


In [7]:
baskets = pd.read_csv(os.path.join(DATA_PATH, 'baskets.csv'), encoding='windows-1251', header=None)

In [8]:
baskets.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,креветки,миндаль,авокадо,овощная смесь,зеленый виноград,цельнозерновая мука,батат,творог,энергетический напиток,томатный сок,низкокалорийный йогурт,зеленый чай,мед,салат,минеральная вода,лосось,ягодный сок,замороженный смузи,шпинат,оливковое масло
1,гамбургер,фрикадельки,яйца,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,чатни,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,индейка,авокадо,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,минеральная вода,молоко,энергетический батончик,рис,зеленый чай,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
baskets.shape

(7501, 20)

In [10]:
baskets_transactions = [list(row.dropna()) for index, row in baskets.iterrows()]

In [30]:
apriori_solver.solve(baskets_transactions, 0.01)

,items_set,support
0,"(макароны, минеральная вода)",0.061192
1,"(минеральная вода, шоколад)",0.052660
2,"(минеральная вода, яйца)",0.050927
3,"(минеральная вода, молоко)",0.047994
4,"(говяжий фарш, минеральная вода)",0.040928
...,...,...
182,"(минеральная вода, хлопья)",0.010265
183,"(гамбургер, рис)",0.010265
184,"(суп, шоколад)",0.010132
185,"(замороженные овощи, низкокалорийный йогурт)",0.010132


In [29]:
min_supports = [0.01, 0.03, 0.05, 0.10, 0.15, 0.3]  # Значения в процентах
execution_times = []  # Для хранения времени выполнения
frequent_item_counts = []  # Для хранения количества частых наборов

# Запуск экспериментов
for min_support in min_supports:
    start_time = time.time()  # Начало замера времени
    apriori_solver = AprioriSolver()  # Инициализация вашего решателя
    # Выполните алгоритм
    result_items = apriori_solver.solve(baskets_transactions, min_support=min_support)  # Измените под свои методы
    exec_time = time.time() - start_time  # Время выполнения

    execution_times.append(exec_time)
    # Подсчет частых наборов
    frequent_item_counts.append(result_items['items_set'].nunique())

In [23]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Добавляем линию для времени выполнения
fig.add_trace(go.Scatter(
    x=min_supports,
    y=execution_times,
    mode='lines+markers',
    name='Время выполнения (сек)',
    yaxis='y1'
))

# Добавляем линию для количества частых наборов
fig.add_trace(go.Scatter(
    x=min_supports,
    y=frequent_item_counts,
    mode='lines+markers',
    name='Количество частых наборов',
    yaxis='y2'
))

# Настройки осей
fig.update_layout(
    title='Анализ времени выполнения и количества частых наборов',
    xaxis_title='Порог поддержки',
    yaxis_title='Время выполнения (сек)',
    yaxis2=dict(
        title='Количество частых наборов',
        overlaying='y',
        side='right'
    ),
)

In [17]:
start_time = time.time() 
time.sleep(1)
exec_time = time.time() - start_time
exec_time

1.0010085105895996